In [1]:
import netCDF4
import csv
import os
import hashlib
import logging
from subprocess import run as srun
from urllib.parse import urlparse
from itertools import product
import time
import numpy as np
import xarray as xr
import rioxarray
import geopandas as gpd
from shapely.geometry import Polygon
import subprocess

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
# def download(uri, ofile, md5):
#     srun(['curl', '-s', '-o', ofile, uri], capture_output=True, check=True)
#     md5dld = str(hashlib.md5(open(ofile, 'rb').read()).hexdigest())
#     if md5 != md5dld:
#         logging.warning("%s != %s", md5, md5dld)
#         logging.info("uri %s (%s == %s) : %s", uri, md5, md5dld, ofile)

In [4]:
# years=[1985]

In [5]:
# models=['EC-Earth3_','TaiESM1']

In [6]:
# scenarios=['historical','ssp126','ssp370']

In [7]:
# variables=['pr','sfcWind']

In [8]:
# future_1=list(range(2025, 2046))

In [9]:
# future_2=list(range(2045, 2066))

In [10]:
# with open('gddp-cmip6-thredds-fileserver.csv') as index:
#     fobjects = csv.reader(index)
#     next(fobjects)
#     for objs in fobjects:
#         md5, uri = [o.strip() for o in objs]
#         prsout = urlparse(uri)
#         ofile = os.path.split(prsout.path)[1]

#         if (any(str(year) in ofile for year in years) and
#             any(model in ofile for model in models) and
#             any(variable in ofile for variable in variables) and
#             any(scenario in ofile for scenario in scenarios)):

#             print(f"Downloading: {ofile}")  # Debug print
#             download(uri, ofile, md5)

**DOWNLOAD RASTER ONLY FOR URUGUAY REGION**

I developed this function to download a dataset, clip it to the geometry of Uruguay, and then remove the original dataset, keeping only the portion that covers Uruguay. The code was adapted from NASA's official website (https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp-cmip6) and from this repository (https://github.com/RaphaelPB/CRVA_tool/blob/main/1-DownloadAndFormatData/Download_NEX-GDDP-CMIP6.py).

In [2]:
def filter_and_download(csv_path, years, models, variables, scenarios, shape):
    """
    Downloads, filters, and processes climate data files based on specific criteria (years, models, variables, scenarios).
    The data is clipped to the Uruguay region, and variables such as precipitation are converted if necessary.

    Parameters:
    -----------
    csv_path : str
        Path to the CSV file containing the index of datasets to download.
    years : list of int
        List of years to filter the datasets.
    models : list of str
        List of model names to filter the datasets.
    variables : list of str
        List of variables (e.g., 'pr', 'tas') to filter the datasets.
    scenarios : list of str
        List of climate scenarios (e.g., 'rcp45', 'rcp85') to filter the datasets.
    shape : geopandas.GeoDataFrame
        A GeoDataFrame containing the geometry (e.g., of Uruguay) to clip the data.
    
    This function downloads files matching the specified criteria, processes them (including clipping to a region),
    and stores the results in NetCDF format, while cleaning up by deleting the original unfiltered files.
    """
    
    with open(csv_path) as index:
        fobjects = csv.reader(index) # Read the CSV file containing dataset metadata
        next(fobjects)  # Skip header

        # Loop through each file entry in the CSV and apply the filtering criteria
        for objs in fobjects:
            md5, uri = [o.strip() for o in objs]

            # Extract the filename from the URL to check against the filtering criteria
            prsout = urlparse(uri)
            ofile = os.path.split(prsout.path)[1]
    
            # Download the file only if it matches the filtering criteria
            if any(str(year) in ofile for year in years) and \
               any(model in ofile for model in models) and \
               any(variable in ofile for variable in variables) and \
               any(scenario in ofile for scenario in scenarios):
        
                print(f"Downloading: {ofile}")  # Print the filename being downloaded for debugging purposes
        
                # Download the file using the provided URI 
                download(uri, ofile, md5)

                filename = os.path.splitext(os.path.basename(uri))[0]
        
                # Open the downloaded dataset using xarray
                data = os.path.join(os.getcwd(), ofile)  # Update with correct path
                ds_GDDP6 = xr.open_dataset(data)
        
                # Set spatial dims and CRS
                ds_GDDP6 = ds_GDDP6.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
                ds_GDDP6.rio.write_crs("epsg:4326", inplace=True)
                ds_GDDP6.coords['lon'] = (ds_GDDP6.coords['lon'] + 180) % 360 - 180
                ds_GDDP6 = ds_GDDP6.sortby(ds_GDDP6.lon)
        
                # Clip the data to the Uruguay region
                ds_GDDP6 = ds_GDDP6.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
                ds_masked = ds_GDDP6.rio.clip(shape.geometry, shape.crs, drop=True)

                if 'pr' in variables:
                    secs_in_a_day = 24 * 60 * 60
                    ds_masked['pr_mmd'] = (ds_masked['pr'] * secs_in_a_day).round(2).astype('float32') # Convert to mm/day
        
                # Save the filtered data to a NetCDF file
                ds_masked.to_netcdf(f"filtered_data/{filename}_masked.nc")
        
                # Delete the original unfiltered file to save space
                os.remove(data)
                print(f"Deleted original unfiltered file: {ofile}")
        
                # Explicitly close datasets to free memory
                ds_GDDP6.close()
                ds_masked.close()

In [12]:
# Define the shapefile for Uruguay area (using GeoPandas)
shape = gpd.read_file("ury_adm_2020_shp.zip")

/opt/conda/lib/python3.11/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'ury_adm_2020_shp.zip': 'ury_admbnda_adm0_2020' (default), 'ury_admbnda_adm1_2020', 'ury_admbnda_adm2_2020', 'ury_admbndl_ALL_2020'. Specify layer parameter to avoid this warning.
  result = read_func(


In [13]:
# Set the filtering parameters
years = list(range(2038, 2039)) #list(range(2055, 2065)) # #list(range(1985, 2015))
models= ['TaiESM1'] #['TaiESM1'] #['EC-Earth3_'] 
variables = ['pr'] #['sfcWind'] #["pr"] #['tas_']
scenarios = ['ssp126'] #["historical"] #['ssp126'] #['ssp370'] # #["historical"]

In [14]:
# Run the process
filter_and_download('gddp-cmip6-thredds-fileserver.csv', years, models, variables, scenarios, shape)

Downloading: pr_day_TaiESM1_ssp126_r1i1p1f1_gn_2038.nc
Deleted original unfiltered file: pr_day_TaiESM1_ssp126_r1i1p1f1_gn_2038.nc
